In [157]:
#!pip3 freeze > "../requirements.txt"
#!pip3 install -r "../requirements.txt"  # giving some error
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import copy
import pickle
import datetime as dt
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from pins import board_rsconnect
from collections import Counter
from IPython.display import display
from datetime import timedelta
import pytz

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 

def GBQ_data(query_string):
    client = bigquery.Client('turing-230020')
    query = client.query(query_string)
    results = query.result()
    return results.to_dataframe()

In [3]:
API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
job_data =board.pin_read("muhammad_usman/demand-basic-data")
print(job_data.shape)

job_data_cols = ['job_id', 'company', 'client_category','opportunity_created_date', 'dc_date', 'max_acceptable_rate', 'is_deleted',
                 'job_value', 'status', 'num_must_have_skills', 'must_have_total_years_of_experience',
                'ms_num_queries', 'ss_queries', 'queries', 'interviews_requested', 'interviews_scheduled',
                'interviews_happened','interviews_passed', 'interviews_failed', 'region', 'engagement_days',
                'number_of_open_roles', 'total_positions', 'total_open_positions__c', 'must_have_skill_ids', 'must_have_skill_names']

final_data = job_data[job_data_cols].copy()
final_data['opportunity_created_date'] = pd.to_datetime(final_data['opportunity_created_date'])
final_data['dc_date'] = pd.to_datetime(final_data['dc_date'])
final_data['dc'] = np.where(final_data['dc_date'].isna(), 0,1)
final_data['ninety_days_old'] = np.where((final_data['opportunity_created_date'] > dt.datetime.now(pytz.utc) - timedelta(days = 90)) & (final_data['dc']==0) , 1,0)
final_data['time_taken_DC'] = (final_data['dc_date'] - final_data['opportunity_created_date']).dt.days
#final_data['time_taken_DC'] = np.where(final_data['time_taken_DC']<0, 0, final_data['time_taken_DC'])
final_data['enterprise'] = np.where(final_data['client_category']=='1.Platinum', 1,0)
final_data['active'] = np.where(~(final_data['status'].isin(['Lost Opportunity', 'Project Aborted', 'Paused'])) & (final_data['dc']==0), 1,0)
final_data['active_roles'] = np.where(final_data['active']==1, final_data['total_positions'],0)
final_data['lost'] = np.where((final_data['status'].isin(['Lost Opportunity', 'Project Aborted', 'Paused']) & (final_data['dc']==0)), 1,0)
final_data['max_acceptable_rate'] = np.where(final_data['max_acceptable_rate'] >300 , 300, final_data['max_acceptable_rate'])
final_data.shape




(9593, 33)

In [69]:
pos = """with pos_data as(SELECT * from(
SELECT *, row_number() over(partition by job_id order by opportunity_created_date desc) as rn
from `matchingmetrics.opps_positions` where opportunity_type='parent') where rn=1)

SELECT * from pos_data
left join turing-dev-337819.pdsa.demand_cluster using(job_id)
"""

pos_data = GBQ_data(pos)
print(pos_data.shape)
pos_data_cols = ['job_id', 'account', 'opportunity_name', 'is_platinum', 'total_positions', 'chosen_sum_in_bulk_job', 'starts_sum_in_bulk_job','client_category', 'cluster']
pos_data = pos_data[pos_data_cols].copy()

pos_data = pos_data.loc[pos_data['account']!='OpenAI'].copy()
pos_data = pos_data.merge(final_data[['job_id', 'active', 'lost', 'num_must_have_skills']], how='left', on='job_id')
pos_data['active_positions'] = np.where(pos_data['lost']==0, pos_data['total_positions']-pos_data['chosen_sum_in_bulk_job'], 0)
print(pos_data.shape)

(7943, 30)
(7913, 14)


## Combination of Skills

In [105]:
API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
job_all_data = board.pin_read("muhammad_usman/demand-avs-data")
job_all_data = job_all_data.loc[~job_all_data['must_have_skill_ids_tuple'].isna()]


job_mcq_data = job_all_data[['job_id', 'must_have_skill_ids', 'must_have_skill_names', 'must_have_skill_ids_tuple', 'passed_must_have_skill total_devs',
                                                     'passed_must_have_skill Elite',
                                                      'passed_must_have_skill High Quality',
                                                      'passed_must_have_skill Average',
                                                      'passed_must_have_skill Low Quality',
                                                      'passed_must_have_skill Low Experience']]

merge_data_pos_mcq = pos_data.merge(job_mcq_data, how='left', on='job_id')
merge_data_pos_mcq = merge_data_pos_mcq.loc[~merge_data_pos_mcq['num_must_have_skills'].isna()]

duck1_comb = merge_data_pos_mcq.groupby(['must_have_skill_ids_tuple','must_have_skill_names']).agg({'num_must_have_skills':'mean',
                                                      'total_positions':'sum', 'chosen_sum_in_bulk_job':'sum',
                                                      'active_positions':'sum',
                                                      'passed_must_have_skill total_devs':'mean',
                                                      'passed_must_have_skill Elite':'mean',
                                                      'passed_must_have_skill High Quality':'mean',
                                                      'passed_must_have_skill Average':'mean',
                                                      'passed_must_have_skill Low Quality':'mean',
                                                      'passed_must_have_skill Low Experience':'mean'}).reset_index()#.to_csv('job_all_data.csv')
duck1_comb.sort_values(by='total_positions', ascending=False)
duck1_comb['sk_li'] = duck1_comb.must_have_skill_ids_tuple.apply(lambda x:x.replace('(', '').replace(')','')).apply(lambda x:x.split(','),-1)
duck1_comb['single_skill'] = ''

## DATA Duplication

In [106]:
global_data = pd.DataFrame()
for x in duck1_comb.index:
    n = int(duck1_comb.at[x,'num_must_have_skills']) #len(duck1_comb.at[x,'sk_li'])
    sk_list = duck1_comb.at[x,'sk_li']
    temp_data = pd.DataFrame(duck1_comb.loc[x,]).transpose()
    temp_data = pd.concat([temp_data]*n, ignore_index=True)
    for y in temp_data.index:
        temp_data.at[y, 'single_skill'] = sk_list[y]
    global_data = global_data.append(temp_data, ignore_index=True)
skill_map = GBQ_data("""SELECT id,skill_name FROM `turing-230020.devdb_mirror.base_all_skills_v4`""")
global_data['single_skill'] = global_data.single_skill.apply(lambda x:int(x))
global_data = global_data.merge(skill_map, how='left', left_on='single_skill', right_on='id')
global_data.to_csv('combination_of_skills.csv')

## Single Skill
- If a job has 2 skills (React, Python) then that job will be considered for both skills.

In [134]:
job_mcq_must = """
SELECT 
    job.job_id, 
    job.skill_id, 
    skill.skill_name
from `turing-230020.devdb_mirror.ms2_job_skill` job
LEFT join `turing-230020.devdb_mirror.base_all_skills_v4` skill on job.skill_id=skill.id
where job.job_skill_level_id=1
"""

job_mcq_must = GBQ_data(job_mcq_must)


ff = pos_data.merge(job_mcq_must, how='left', on='job_id')
ff = ff.loc[~ff['num_must_have_skills'].isna()]
ff = ff.groupby(['skill_id', 'skill_name']).agg({'total_positions':'sum','chosen_sum_in_bulk_job':'sum','active_positions':'sum'}).reset_index()


In [135]:

API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
dev_data = board.pin_read("muhammad_usman/devs-skill-pass-list")
dev_data['last_update_availability'] = pd.to_datetime(dev_data['last_update_availability'])
dev_data = dev_data.loc[(dev_data['num_passed_skill']>0) & (~dev_data['num_skill'].isna())]
dev_data = dev_data.loc[(dev_data['vetting_status'].isin(['available supply','ready 2.0','passed ti','standardized resume']))].reset_index(drop=True).copy()
dev_data['self_skill_id_list'] = dev_data.self_skill_id.apply(lambda x:list(map(int, x.split(',',-1))))
dev_data['passed_skill_id_list'] = dev_data.passed_skill_id.apply(lambda x:list(map(int, x.replace('[', '').replace(']','').split(',',-1))))
dev_data['passed_skill_id_set'] = dev_data.passed_skill_id_list.apply(lambda x:set(x))
dev_data['self_skill_id_set'] = dev_data.self_skill_id_list.apply(lambda x:set(x))
dev_data.shape

def avs_supply_passed_mcq(skill):
    temp_data = dev_data                            
    for s in skill:
        temp_data = temp_data.loc[temp_data['passed_skill_id_set'].apply(lambda x:s in x)]
    return temp_data['cluster_label'].value_counts().append(pd.Series(temp_data.shape[0], index = ['total_devs']))

col_name = ['total_devs', 'Elite', 'High Quality', 'Average', 'Low Quality', 'Low Experience']
df_passed = pd.DataFrame(columns = col_name)
for i in ff.skill_id:
    df_passed = df_passed.append(avs_supply_passed_mcq([i]),ignore_index=True)

df_passed['skill_id'] = ff.skill_id
ff.merge(df_passed, how='left', on='skill_id').to_csv('without_openai_active.csv')


In [147]:
#ff.merge(df_passed, how='left', on='skill_id').sort_values(by='total_positions', ascending=False)
#ff.merge(df_passed, how='left', on='skill_id').isnull().sum()